<a href="https://colab.research.google.com/github/Kathy42xu/DL_TA/blob/main/Autocoder_on_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data preperation


In [ ]:
import os

os.makedirs('movielens', exist_ok=True)


!wget -O movielens/ml-32m.zip https://files.grouplens.org/datasets/movielens/ml-32m.zip

!unzip movielens/ml-32m.zip -d movielens/

import pandas as pd

# Load the ratings data
ratings = pd.read_csv('movielens/ml-32m/ratings.csv')
print("Ratings Data:")
print(ratings.head())

# Load the movies data
movies = pd.read_csv('movielens/ml-32m/movies.csv')
print("\nMovies Data:")
print(movies.head())

# Load the tags data
tags = pd.read_csv('movielens/ml-32m/tags.csv')
print("\nTags Data:")
print(tags.head())

# Load the links data
links = pd.read_csv('movielens/ml-32m/links.csv')
print("\nLinks Data:")
print(links.head())


--2025-01-11 23:58:07--  https://files.grouplens.org/datasets/movielens/ml-32m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238950008 (228M) [application/zip]
Saving to: ‘movielens/ml-32m.zip’

movielens/ml-32m.zi 100%[===================>] 227.88M  71.2MB/s    in 3.2s    

2025-01-11 23:58:11 (71.2 MB/s) - ‘movielens/ml-32m.zip’ saved [238950008/238950008]

Archive:  movielens/ml-32m.zip
   creating: movielens/ml-32m/
  inflating: movielens/ml-32m/tags.csv  
  inflating: movielens/ml-32m/links.csv  
  inflating: movielens/ml-32m/README.txt  
  inflating: movielens/ml-32m/checksums.txt  
  inflating: movielens/ml-32m/ratings.csv  
  inflating: movielens/ml-32m/movies.csv  
Ratings Data:
   userId  movieId  rating  timestamp
0       1       17     4.0  944249077
1       1       25     1.0  944250228
2       1       

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load datasets
ratings = pd.read_csv('movielens/ml-32m/ratings.csv')
movies = pd.read_csv('movielens/ml-32m/movies.csv')

# Merge ratings with movies to filter based on titles
merged_data = ratings.merge(movies, on="movieId")
all_ratings = merged_data[
    merged_data['title'].str.endswith(tuple(f"({year})" for year in range(2020, 2025)))
]

# Display filtered data
print(f"Amount of data after filtering: {len(all_ratings)}")
print(all_ratings.head())

# Create user-item interaction matrix
user_item_matrix = all_ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Split into train and test sets
train_data, test_data = train_test_split(user_item_matrix, test_size=0.2, random_state=42)


Amount of data after filtering: 273932
      userId  movieId  rating   timestamp  \
2237      22   235509     2.5  1611070726   
2238      22   247150     3.0  1622483323   
2239      22   278420     4.0  1668895485   
2240      22   279412     3.5  1668895505   
2241      22   280236     4.0  1674334098   

                                    title                   genres  
2237              Outside the Wire (2021)   Action|Sci-Fi|Thriller  
2238                      Stowaway (2021)             Drama|Sci-Fi  
2239  Weird: The Al Yankovic Story (2022)             Comedy|Drama  
2240                Enola Holmes 2 (2022)  Adventure|Crime|Mystery  
2241                      Spirited (2022)                   Comedy  


#Build auto encoder

In [ ]:
import tensorflow as tf

class AutoRec:
    def __init__(self, num_items, hidden_neurons, learning_rate, lambda_value):
        self.num_items = num_items
        self.hidden_neurons = hidden_neurons
        self.learning_rate = learning_rate
        self.lambda_value = lambda_value

        self.input_R = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, self.num_items])
        self.input_mask_R = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, self.num_items])

        self.build_model()

    def build_model(self):
      with tf.compat.v1.variable_scope("AutoRec", reuse=tf.compat.v1.AUTO_REUSE):
        V = tf.compat.v1.get_variable("V", shape=[self.num_items, self.hidden_neurons],
                                      initializer=tf.compat.v1.truncated_normal_initializer(mean=0, stddev=0.03))
        W = tf.compat.v1.get_variable("W", shape=[self.hidden_neurons, self.num_items],
                                      initializer=tf.compat.v1.truncated_normal_initializer(mean=0, stddev=0.03))
        mu = tf.compat.v1.get_variable("mu", initializer=tf.zeros(self.hidden_neurons))
        b = tf.compat.v1.get_variable("b", initializer=tf.zeros(self.num_items))

        # Encoder
        self.encoder = tf.nn.sigmoid(tf.matmul(self.input_R, V) + mu)

        # Decoder
        self.decoder = tf.matmul(self.encoder, W) + b

        # Loss
        pre_rec_cost = tf.multiply(self.input_R - self.decoder, self.input_mask_R)
        rec_cost = tf.reduce_sum(tf.square(pre_rec_cost))
        reg_cost = self.lambda_value * 0.5 * (tf.nn.l2_loss(W) + tf.nn.l2_loss(V))

        self.loss = rec_cost + reg_cost
        self.optimizer = tf.compat.v1.train.AdamOptimizer(self.learning_rate).minimize(self.loss)


    def train(self, train_data, mask, epochs, batch_size):
        session = tf.compat.v1.Session()
        session.run(tf.compat.v1.global_variables_initializer())

        for epoch in range(epochs):
            total_loss = 0
            for i in range(0, len(train_data), batch_size):
                batch_data = train_data[i:i + batch_size]
                batch_mask = mask[i:i + batch_size]
                feed_dict = {self.input_R: batch_data, self.input_mask_R: batch_mask}
                _, batch_loss = session.run([self.optimizer, self.loss], feed_dict=feed_dict)
                total_loss += batch_loss
            print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_data)}")

        self.session = session

    def predict(self, test_data):
        feed_dict = {self.input_R: test_data}
        return self.session.run(self.decoder, feed_dict=feed_dict)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Prepare mask for training
train_mask = (train_data > 0).astype(float)

# Instantiate and train AutoRec
num_items = user_item_matrix.shape[1]
hidden_neurons = 500
learning_rate = 0.001
lambda_value = 0.1
batch_size = 128
epochs = 50

autorec = AutoRec(num_items, hidden_neurons, learning_rate, lambda_value)
autorec.train(train_data.values, train_mask.values, epochs, batch_size)

# Predict on test set
test_predictions = autorec.predict(test_data.values)

# Evaluate
test_mask = (test_data > 0).astype(float)
true_ratings = test_data.values[test_mask.values > 0]
predicted_ratings = test_predictions[test_mask.values > 0]

rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
mae = mean_absolute_error(true_ratings, predicted_ratings)

print(f"RMSE: {rmse}, MAE: {mae}")


Epoch 1, Loss: 26.490068384038864
Epoch 2, Loss: 12.821944492805994
Epoch 3, Loss: 11.010856953514562
Epoch 4, Loss: 9.72670793378929
Epoch 5, Loss: 8.631821431670222
Epoch 6, Loss: 7.528749337237728
Epoch 7, Loss: 6.725265046411066
Epoch 8, Loss: 6.018965935679301
Epoch 9, Loss: 5.430313302049858
Epoch 10, Loss: 4.908460137078589
Epoch 11, Loss: 4.491645234928988
Epoch 12, Loss: 4.083661553094214
Epoch 13, Loss: 3.7638386050865233
Epoch 14, Loss: 3.448445496388072
Epoch 15, Loss: 3.1918452568829885
Epoch 16, Loss: 2.9980573808570985
Epoch 17, Loss: 2.9115718806473923
Epoch 18, Loss: 2.804651219615385
Epoch 19, Loss: 2.6664899409303637
Epoch 20, Loss: 2.5349965722368504
Epoch 21, Loss: 2.4746736670510012
Epoch 22, Loss: 2.4151068525508252
Epoch 23, Loss: 2.4064883293998687
Epoch 24, Loss: 2.4228810613103797
Epoch 25, Loss: 2.489017366513797
Epoch 26, Loss: 2.545472761553629
Epoch 27, Loss: 2.5603888948537143
Epoch 28, Loss: 2.5026590223526064
Epoch 29, Loss: 2.415405444384949
Epoch 30,